# SharePoint

## Install Packages

In [1]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import the libraries

In [13]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import SharepointTool, MessageRole

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [14]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Setup the SharePoint Tool

In [ ]:
# Enter SharePoint connection name here
sharepoint_connection = "sharepoint_data"  # Replace with your SharePoint connection name

# Extract the connection list.
conn_list = project_client.connections.list()
sharepoint_connection_id = ""

# Find the connection IDs for SharePoint
for conn in conn_list:
    if conn.name == sharepoint_connection:
        sharepoint_connection_id = conn.id


# Print the connection names
print(f"SharePoint Connection ID: {sharepoint_connection_id}")

# Initialize the SharePoint tools
sharepoint_tool = SharepointTool(connection_id=sharepoint_connection_id)

SharePoint Connection ID: /subscriptions/73047bc3-f020-4930-b16d-fe5c9dd3c73e/resourceGroups/agent-standard/providers/Microsoft.CognitiveServices/accounts/foundrydlt4/projects/projectdlt4/connections/sharepoint_data


## Create the agent

In [16]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-sharepoint-agent",  # Name of the agent
    instructions="You are a helpful agent and can search information from uploaded files",  # Instructions for the agent
    tools=sharepoint_tool.definitions
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_gzgp65UYyglVjfJc2YBNFc9e


## Create thread

In [17]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_560maQBk3JURrA33BItPC9JS


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [18]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Run the agent

In [19]:
user_input = "Provide details on the Northwind Standard Plan"
run_agent(user_input)

Created message, ID: msg_xRHa3FatP4uboGyxrsvRC7Pj
Run finished with status: completed
Agent Response: The Northwind Standard health plan offers comprehensive coverage at an affordable cost. Here are the key details:

### Costs
- **Premiums**: Paid monthly and based on age, health, and type of coverage.
- **Deductibles**: $2,000 per person per year.
- **Co-payments and Co-insurance**:
  - $30 for primary care visits.
  - $50 for specialist visits.
  - 20% co-insurance after deductible is met.
- **Out-of-Pocket Maximum**: $6,000 per person per year.

### Covered Services
- **Hospitals**: Access to various in-network hospitals, comprehensive maternity care including prenatal, delivery, and post-delivery care.
- **Prescription Drugs**:
  - Coverage for both brand name and generic medications.
  - Must use preferred pharmacies for lowest copayments.
  - Offers mail delivery and online management of prescriptions.
  - Excludes drugs used for weight loss or fertility treatments, compounded me

In [20]:
user_input = "Provide a tabular comparison between the Northwind Standard and Northwind Healthplus plans"
run_agent(user_input)

Created message, ID: msg_V8MREI2MFa7WQDeN9kGmIbfR
Run finished with status: completed
Agent Response: Here is a tabular comparison of the Northwind Standard and Northwind Healthplus plans based on the extracted details:

| Feature                         | Northwind Standard                        | Northwind Healthplus                        |
|---------------------------------|-------------------------------------------|--------------------------------------------|
| **Premiums**                    | Monthly, based on age and health          | Varies by plan type and services used      |
| **Deductibles**                 | $2,000 per person per year                | Not specified                              |
| **Co-payments**                 | $30 for primary care, $50 for specialists | Not specified                              |
| **Co-insurance**                | 20% after deductible                      | Not specified                              |
| **Out-of-Pocket Maximum** 

## Cleanup resources

In [21]:
# Cleanup resources
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")


Deleted agent
